# Train and Host a Keras Sequential Model

This notebook shows how to train and host a Keras Sequential model on SageMaker. The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

## Set up the environment

In [11]:
import os
import sagemaker
from sagemaker import get_execution_role
import boto3

sagemaker_session = sagemaker.Session()

role = ""

## Download the CIFAR-10 dataset
Downloading the test and training data will take around 5 minutes.

In [13]:
!cat utils.py

import os
import sys
import tarfile
import boto3
from six.moves import urllib
from ipywidgets import FloatProgress
from IPython.display import display


def cifar10_download(data_dir='/tmp/cifar10_data', print_progress=True):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if os.path.exists(os.path.join(data_dir, 'cifar-10-batches-bin')):
        print('cifar dataset already downloaded')
        return

    filename = 'cifar-10-binary.tar.gz'
    filepath = os.path.join(data_dir, filename)

    if not os.path.exists(filepath):
        region = boto3.Session().region_name
        boto3.Session().resource('s3', region_name=region).Bucket('sagemaker-sample-data-{}'.format(region)).download_file('tensorflow/cifar10/cifar-10-binary.tar.gz', '/tmp/cifar10_data/cifar-10-binary.tar.gz')

    tarfile.open(filepath, 'r:gz').extractall(data_dir)


In [ ]:
import utils

utils.cifar10_download()

## Upload the dataset to an S3 bucket

In [2]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

In [ ]:
print(inputs)

`sagemaker_session.upload_data` will upload the CIFAR-10 dataset from this machine to a bucket named **sagemaker-{region}-{*your aws account number*}**, if you don't have this bucket yet, `sagemaker_session` will create it for you.

## Complete source code
Here is the full source code for the model:

In [8]:
!cat cifar10_cnn.py

#     Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#     Licensed under the Apache License, Version 2.0 (the "License").
#     You may not use this file except in compliance with the License.
#     A copy of the License is located at
#    
#         https://aws.amazon.com/apache-2-0/
#    
#     or in the "license" file accompanying this file. This file is distributed
#     on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#     express or implied. See the License for the specific language governing
#     permissions and limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import tensorflow as tf
from tensorflow.python.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.saved_model.signature_constants import PREDICT_INPUTS
from tensor

Lets take a closer look:

### The model function
This function constitutes the main difference between TensorFlow and Keras models on SageMaker; Keras models have a `keras_model_fn`:

In [ ]:
def keras_model_fn(hyperparameters):
    """keras_model_fn receives hyperparameters from the training job and returns a compiled keras model.
    The model will be transformed into a TensorFlow Estimator before training and it will be saved in a 
    TensorFlow Serving SavedModel at the end of training.

    Args:
        hyperparameters: The hyperparameters passed to the SageMaker TrainingJob that runs your TensorFlow 
                         training script.
    Returns: A compiled Keras model
    """
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', name='inputs', input_shape=(HEIGHT, WIDTH, DEPTH)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    opt = RMSPropOptimizer(learning_rate=hyperparameters['learning_rate'], decay=hyperparameters['decay'])

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

This function builds and returns a compiled Keras model.

**Note:** The first layer is named `PREDICT_INPUTS`. This serves as a workaround for a known issue where TensorFlow does not recognize the default (or any custom) name for the first layer of Keras models. Furthermore, note that we are wrapping our model in a `tf.keras.Model` before returning it. This serves as a workaround for a known issue where a Sequential model cannot be directly converted into an Estimator. See [here](https://github.com/tensorflow/tensorflow/issues/20552) for more information about the issue.

### Input functions
These functions are similar to those required by any other model using the TensorFlow Estimator API.

In [ ]:
def serving_input_fn(params):
    # Notice that the input placeholder has the same input shape as the Keras model input
    tensor = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, DEPTH])
    
    # The inputs key INPUT_TENSOR_NAME matches the Keras InputLayer name
    inputs = {INPUT_TENSOR_NAME: tensor}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


def train_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.TRAIN,
                    batch_size=BATCH_SIZE, data_dir=training_dir)


def eval_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.EVAL,
                    batch_size=BATCH_SIZE, data_dir=training_dir)

The `train_` and `eval_` functions call the `_input` function which returns a properly processed and shuffled (for training) set of images and labels.

## Create a training job using the SageMaker TensorFlow Estimator

In [7]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar10_cnn.py',
                       role=role,
                       framework_version='1.12.0',
                       hyperparameters={'learning_rate': 1e-4, 'decay':1e-6},
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=2, train_instance_type='ml.c4.xlarge')

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-2019-03-21-17-28-50-058


2019-03-21 17:28:51 Starting - Starting the training job...
2019-03-21 17:28:53 Starting - Launching requested ML instances......
2019-03-21 17:29:55 Starting - Preparing the instances for training...
2019-03-21 17:30:55 Downloading - Downloading input data
2019-03-21 17:30:55 Training - Downloading the training image..
2019-03-21 17:30:59,725 INFO - root - running container entrypoint
2019-03-21 17:30:59,725 INFO - root - starting train task
2019-03-21 17:30:59,746 INFO - container_support.training - Training starting
2019-03-21 17:31:02,656 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2019-03-21 17:31:02,656 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2019-03-21 17:31:02,656 INFO - tf_container - ---------------------------------------------------------
2019-03-21 17:31:02,656 INFO - tf_container - creating RunConfig:
2019-03-21 17:31:02,656 INFO - tf_container - {'sa

2019-03-21 17:35:11,872 INFO - tensorflow - Evaluation [40/100]
2019-03-21 17:35:12,625 INFO - tensorflow - Evaluation [50/100]
2019-03-21 17:35:13,434 INFO - tensorflow - Evaluation [60/100]
2019-03-21 17:35:14,187 INFO - tensorflow - Evaluation [70/100]
2019-03-21 17:35:14,878 INFO - tensorflow - Finished evaluation at 2019-03-21-17:35:14
2019-03-21 17:35:14,878 INFO - tensorflow - Saving dict for global step 1000: accuracy = 0.44214794, global_step = 1000, loss = 1.56057
2019-03-21 17:35:15,298 INFO - tensorflow - Saving 'checkpoint_path' summary for global step 1000: s3://sagemaker-eu-west-1-526952723483/sagemaker-tensorflow-2019-03-21-17-28-50-058/checkpoints/model.ckpt-1000
2019-03-21 17:35:15,641 INFO - tensorflow - Calling model_fn.
2019-03-21 17:35:15,741 INFO - tensorflow - Done calling model_fn.
2019-03-21 17:35:15,741 INFO - tensorflow - Signatures INCLUDED in export for Eval: None
2019-03-21 17:35:15,741 INFO - tensorflow - Signatures INCLUDED in export for Classify: None


**Note**: Keras models have a known issue and cannot be used for distributed (multi-instance) training. Keep `train_instance_count == 1` until the TensorFlow/Keras team support this feature. See [here](https://github.com/tensorflow/tensorflow/issues/14504) for more information about the issue.


## Deploy the trained model

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [9]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-2019-03-21-17-28-50-058
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2019-03-21-17-28-50-058


---------------------------------------------------------------------------!

## Make some predictions
Prediction is not the focus of this notebook, so to verify the endpoint's functionality, we'll simply generate random data in the correct shape and make a prediction.

In [14]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)

# The inputs key 'inputs_input' matches the Keras InputLayer name
predictor.predict({'inputs_input': data}) 

{'outputs': {'activation_5': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 10}]},
   'float_val': [0.004174378234893084,
    0.4450441002845764,
    0.0013531706063076854,
    0.004747383296489716,
    0.0019372832030057907,
    0.001180722494609654,
    0.035327520221471786,
    0.010748269036412239,
    0.0006896215490996838,
    0.4947974979877472]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1553189715},
  'signature_name': 'serving_default'}}

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the SageMaker Endpoint:

In [15]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-tensorflow-2019-03-21-17-28-50-058
